In [1]:
import sys
v_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'
if v_path in sys.path:
    sys.path.remove(v_path)
import numpy as np
import pandas as pd
from keras.applications.resnet50 import ResNet50
from keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D, Concatenate, UpSampling2D, Input
from PIL import Image  
import os
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
data = pd.read_csv('data1.csv')

In [3]:
def make_square(fp):
    image = Image.open(fp)  
    image = image.resize((224, 224))
    r_img = np.array(image)
    if(r_img.shape == (224, 224)):
        return r_img.reshape(224, 224, 1)
    return r_img[:,:,0].reshape(224, 224, 1)

In [4]:
data

,im1,im2,seg1,seg2,seg3,seg4,seg5,seg6,seg7,seg8,label
0,Training/Damaged/ID (1)/AP/AP.jpg,Training/Damaged/ID (1)/LAT/LAT.jpg,Training/Damaged/ID (1)/AP/Ap_Pedicle.png,Training/Damaged/ID (1)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (1)/AP/Ap_Vertebra.png,Training/Damaged/ID (1)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (1)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (1)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (1)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (1)/LAT/Lat_Vertebra.png,0
1,Training/Damaged/ID (2)/AP/AP.jpg,Training/Damaged/ID (2)/LAT/LAT.jpg,Training/Damaged/ID (2)/AP/Ap_Pedicle.png,Training/Damaged/ID (2)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (2)/AP/Ap_Vertebra.png,Training/Damaged/ID (2)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (2)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (2)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (2)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (2)/LAT/Lat_Vertebra.png,0
2,Training/Damaged/ID (3)/AP/AP.jpg,Training/Damaged/ID (3)/LAT/LAT.jpg,Training/Damaged/ID (3)/AP/Ap_Pedicle.png,Training/Damaged/ID (3)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (3)/AP/Ap_Vertebra.png,Training/Damaged/ID (3)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (3)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (3)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (3)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (3)/LAT/Lat_Vertebra.png,0
3,Training/Damaged/ID (4)/AP/AP.jpg,Training/Damaged/ID (4)/LAT/LAT.jpg,Training/Damaged/ID (4)/AP/Ap_Pedicle.png,Training/Damaged/ID (4)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (4)/AP/Ap_Vertebra.png,Training/Damaged/ID (4)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (4)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (4)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (4)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (4)/LAT/Lat_Vertebra.png,0
4,Training/Damaged/ID (5)/AP/AP.jpg,Training/Damaged/ID (5)/LAT/LAT.jpg,Training/Damaged/ID (5)/AP/Ap_Pedicle.png,Training/Damaged/ID (5)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (5)/AP/Ap_Vertebra.png,Training/Damaged/ID (5)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (5)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (5)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (5)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (5)/LAT/Lat_Vertebra.png,0
5,Training/Damaged/ID (6)/AP/AP.jpg,Training/Damaged/ID (6)/LAT/LAT.jpg,Training/Damaged/ID (6)/AP/Ap_Pedicle.png,Training/Damaged/ID (6)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (6)/AP/Ap_Vertebra.png,Training/Damaged/ID (6)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (6)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (6)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (6)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (6)/LAT/Lat_Vertebra.png,0
6,Training/Damaged/ID (7)/AP/AP.jpg,Training/Damaged/ID (7)/LAT/LAT.jpg,Training/Damaged/ID (7)/AP/Ap_Pedicle.png,Training/Damaged/ID (7)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (7)/AP/Ap_Vertebra.png,Training/Damaged/ID (7)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (7)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (7)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (7)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (7)/LAT/Lat_Vertebra.png,0
7,Training/Damaged/ID (8)/AP/AP.jpg,Training/Damaged/ID (8)/LAT/LAT.jpg,Training/Damaged/ID (8)/AP/Ap_Pedicle.png,Training/Damaged/ID (8)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (8)/AP/Ap_Vertebra.png,Training/Damaged/ID (8)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (8)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (8)/LAT/Lat_Posterior_Vert...,Training/Damaged/ID (8)/LAT/Lat_Spinous_Proces...,Training/Damaged/ID (8)/LAT/Lat_Vertebra.png,0
8,Training/Damaged/ID (9)/AP/AP.jpg,Training/Damaged/ID (9)/LAT/LAT.jpg,Training/Damaged/ID (9)/AP/Ap_Pedicle.png,Training/Damaged/ID (9)/AP/Ap_Spinous_Process.png,Training/Damaged/ID (9)/AP/Ap_Vertebra.png,Training/Damaged/ID (9)/LAT/Lat_Anterior_Verte...,Training/Damaged/ID (9)/LAT/Lat_Disk_Height.png,Training/Damaged/ID (9)/

In [5]:
X = []
y = []
y_seg1 = []
y_seg2 = []
y_seg3 = []
y_seg4 = []
y_seg5 = []
y_seg6 = []
y_seg7 = []
y_seg8 = []

for i in range(data.shape[0]):
    print(i)
    fp1 = data['im1'][i]
    fp2 = data['im2'][i]
    
    seg_fp1 = data['seg1'][i]
    seg_fp2 = data['seg2'][i]
    seg_fp3 = data['seg3'][i]
    seg_fp4 = data['seg4'][i]
    seg_fp5 = data['seg5'][i]
    seg_fp6 = data['seg6'][i]
    seg_fp7 = data['seg7'][i]
    seg_fp8 = data['seg8'][i]
    
    if(os.path.exists(fp1) and os.path.exists(fp2)
       and os.path.exists(seg_fp1) and os.path.exists(seg_fp2) and os.path.exists(seg_fp3)
       and os.path.exists(seg_fp4) and os.path.exists(seg_fp5) and os.path.exists(seg_fp6)
       and os.path.exists(seg_fp7) and os.path.exists(seg_fp8)):
        image1 = make_square(fp1)
        image2 = make_square(fp2)
        seg1_img = make_square(seg_fp1)
        seg2_img = make_square(seg_fp2)
        seg3_img = make_square(seg_fp3)
        seg4_img = make_square(seg_fp4)
        seg5_img = make_square(seg_fp5)
        seg6_img = make_square(seg_fp6)
        seg7_img = make_square(seg_fp7)
        seg8_img = make_square(seg_fp8)
        datapoint = np.append(image1, image2, axis=2)
        X.append(datapoint)
        y_seg1.append(seg1_img)
        y_seg2.append(seg2_img)
        y_seg3.append(seg3_img)
        y_seg4.append(seg4_img)
        y_seg5.append(seg5_img)
        y_seg6.append(seg6_img)
        y_seg7.append(seg7_img)
        y_seg8.append(seg8_img)
        y.append(data['label'][i])
X = np.array(X)
y = np.array(y)
y_seg1 = np.array(y_seg1)
y_seg2 = np.array(y_seg2)
y_seg3 = np.array(y_seg3)
y_seg4 = np.array(y_seg4)
y_seg5 = np.array(y_seg5)
y_seg6 = np.array(y_seg6)
y_seg7 = np.array(y_seg7)
y_seg8 = np.array(y_seg8)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
X0 = np.zeros((X.shape[0], 224, 224)).reshape(X.shape[0], 224, 224, 1)
X = np.append(X, X0, axis=3)


In [7]:
y_seg1.shape

(676, 224, 224, 1)

In [8]:
y_seg = np.append(y_seg1, y_seg2, axis=3)
y_seg = np.append(y_seg, y_seg3, axis=3)
y_seg = np.append(y_seg, y_seg4, axis=3)
y_seg = np.append(y_seg, y_seg5, axis=3)
y_seg = np.append(y_seg, y_seg6, axis=3)
y_seg = np.append(y_seg, y_seg7, axis=3)
y_seg = np.append(y_seg, y_seg8, axis=3)

In [9]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = UpSampling2D((2, 2))(x)
    concat = Concatenate()([us, skip])
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [10]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = Input((224, 224, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = Conv2D(8, (1, 1), padding="same", activation="sigmoid")(u4)
    model = Model(inputs, outputs)
    return model

In [11]:
from keras import backend as K
def dice_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
    return dice
def dice_coef_loss(y_true, y_pred, smooth=1):
    return -dice_coef(y_true, y_pred)

In [12]:
model = UNet()
model.compile(optimizer="adam", loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 224, 224, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 16) 2320        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 112, 112, 16) 0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [ ]:
model.fit(X, y_seg, validation_split=0.2, epochs=40, batch_size=16, shuffle=True)

In [ ]:
y_pred = model.predict(X)